# Generate graph for multiple benign and attacked imgs

## Import packages

In [30]:
# Set auto reload for python functions
%reload_ext autoreload
%autoreload 2

# Helper libraries
import os
import cv2
import glob
import tqdm
import json
import numpy as np
from time import time
import tensorflow as tf
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

# GoogLeNet
import lucid.modelzoo.vision_models as models

# Libraries provided by Massif project
import constant
import A_matrix
import I_matrix
import model_helper
import gen_graph

print(tf.VERSION)

1.15.0


## Import GoogLeNet (InceptionV1) model

In [3]:
googlenet = models.InceptionV1()
googlenet.load_graphdef()

## Get constants and hyperparameters

In [32]:
args = constant.Args
arg_keys = [arg for arg in args.__dict__.keys() if '__' not in arg]

print('Constants and hyperparameters in args:')
print(arg_keys)

Constants and hyperparameters in args:
['gpu', 'batch_A', 'batch_I', 'layer', 'k_A', 'layer_sizes', 'layers', 'layer_blk_sizes', 'blk_headers', 'num_classes', 'img_width', 'img_height']


## Read benign images

In [6]:
benign_dirpath = '../data/sample-images/sample-benign'
correct_classname = 'giant_panda'

benign_imgs = []
for benign_filename in os.listdir(benign_dirpath):
    if 'jpg' in benign_filename:
        benign_img = plt.imread('{}/{}'.format(benign_dirpath, benign_filename))
        benign_imgs.append(benign_img)

## Read attacked images with different strength

In [9]:
epss = [0.5, 1, 1.5, 2, 2.5, 3, 3.5]
attacked_imgs = {eps: [] for eps in epss}

for eps in epss:
    attacked_dirpath = '../data/sample-images/attacked-{}'.format(eps)
    for attacked_filename in os.listdir(attacked_dirpath):        
        if 'npy' in attacked_filename:
            img = np.load('{}/{}'.format(attacked_dirpath, attacked_filename))
            attacked_imgs[eps].append(img)

## A_matrix: generate nodes

### A_matrix of the **benign** images

In [13]:
# Initialize A-matrix
A_benign = A_matrix.init_A_matrix_single_class(args)

# Get activation score of all neurons in all layers for all images
act_scores_benign = model_helper.get_all_layers_activation_score(googlenet, benign_imgs, args.layers)

# Generate A-matrix based on the activatin score
for layer in args.layers:
    median_act_score_across_imgs = np.median(act_scores_benign[layer], axis=0)
    A_benign[layer] = median_act_score_across_imgs

In [15]:
# Get string converted A-matrix to save it into json file
str_A_benign = {layer: A_benign[layer].astype(str).tolist() for layer in args.layers}

# Save the string converted A-matrix into json file
targeted_classname = 'armadillo'
A_matrix_benign_dirpath = '/Users/haekyu/data/massif/A-mat/aggregated/benign'
filename = '{}/A-benign-{}-{}.json'.format(A_matrix_benign_dirpath, correct_classname, targeted_classname)
with open (filename, 'w') as f:
    json.dump(str_A_benign, f) 

### A_matrix of the **attacked** image with different strength

In [16]:
A_attacked_dict = {}
for eps in epss:
    A_attacked = A_matrix.init_A_matrix_single_class(args)
    A_attacked_dict[eps] = A_attacked

In [17]:
# generate A-matrix for atttacked images with different eps
for eps in epss:
    
    # Get the activation scores
    act_scores_attacked = model_helper.get_all_layers_activation_score(googlenet, attacked_imgs[eps], args.layers)
    
    # Save the median activation scores
    for layer in args.layers:
        median_act_score_across_imgs = np.median(act_scores_attacked[layer], axis=0)
        A_attacked_dict[eps][layer] = median_act_score_across_imgs

In [18]:
# Get string converted A-matrix to save it into json file
str_A_attacked_dict = {eps: {layer: A_attacked_dict[eps][layer].astype(str).tolist() for layer in args.layers} for eps in epss}

# Save the string converted A-matrix
A_matrix_attacked_dirpath = '/Users/haekyu/data/massif/A-mat/aggregated/attacked'
for eps in epss:
    filename = '{}/A-attacked-{}-{}-{}.json'.format(A_matrix_attacked_dirpath, correct_classname, targeted_classname, eps)
    with open (filename, 'w') as f:
        json.dump(str_A_attacked_dict[eps], f) 

## I_matrix: generate edges

### I_matrix of the **benign** images

In [20]:
# Generate I matrix for benign images
I_benign = I_matrix.gen_aggregated_I_matrix(args, benign_imgs, googlenet)

In [21]:
# Convert I matrix into string to save the matrix into json file
blks = list(I_benign.keys())
str_I_benign = {blk: I_benign[blk].astype(str).tolist() for blk in blks}

# Save the string converted I-matrix into json file
I_matrix_benign_dirpath = '/Users/haekyu/data/massif/I-mat/aggregated/benign'
filename = '{}/I-benign-{}-{}.json'.format(I_matrix_benign_dirpath, correct_classname, targeted_classname)
with open (filename, 'w') as f:
    json.dump(str_I_benign, f) 

### I_matrix of the **attacked** images with different strength

In [22]:
Is_attacked = {}
for eps in epss:
    I_attacked = I_matrix.gen_aggregated_I_matrix(args, attacked_imgs[eps], googlenet)
    Is_attacked[eps] = I_attacked

In [35]:
# Save I matrix for attacked images with different eps
I_matrix_attacked_dirpath = '/Users/haekyu/data/massif/I-mat/aggregated/attacked'

for eps in epss:
    filename = '{}/I-attacked-{}-{}-{}.json'.format(I_matrix_attacked_dirpath, correct_classname, targeted_classname, eps)

    # Convert I matrix into string
    str_I_attacked = {blk: Is_attacked[eps][blk].astype(str).tolist() for blk in blks}
    
    # Save the string converted I-matrix into json file
    with open (filename, 'w') as f:
        json.dump(str_I_attacked, f) 

### [No Need] Generate full graph of the **Benign** images 

In [41]:
G_benign = gen_graph.gen_full_graph(args, A_benign, I_benign)

In [42]:
G_benign_json = json_graph.node_link_data(G_benign)

In [43]:
benign_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_benign_json['nodes']))
benign_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_benign_json['links']))
G_benign_str_json = {'nodes': benign_str_parsed_nodes, 'links': benign_str_parsed_links}

In [44]:
graph_dir_path = '/Users/haekyu/data/massif/full-graph/aggregated'
file_path = '{}/{}.json'.format(graph_dir_path, 'G-benign')
with open(file_path, 'w') as f:
    json.dump(G_benign_str_json, f)

### [No Need] Generate full graph of the **Attacked** images with different strength

In [45]:
for eps in epss:
    G_attacked = gen_graph.gen_full_graph(args, A_attacked_dict[eps], Is_attacked[eps])
    G_attacked_json = json_graph.node_link_data(G_attacked)
    attacked_str_parsed_nodes = list(map(lambda x: {'weight': str(x['weight']), 'id': x['id']}, G_attacked_json['nodes']))
    attacked_str_parsed_links = list(map(lambda x: {'source': x['source'], 'target': x['target'], 'weight': str(x['weight'])}, G_attacked_json['links']))
    G_attacked_str_json = {'nodes': attacked_str_parsed_nodes, 'links': attacked_str_parsed_links}
    graph_dir_path = '/Users/haekyu/data/massif/full-graph/aggregated'
    file_path = '{}/{}-{}.json'.format(graph_dir_path, 'G-attacked', eps)
    with open(file_path, 'w') as f:
        json.dump(G_attacked_str_json, f)

## Get the top neurons in **Benign** graph

In [59]:
top_p = 0.2
top_neurons_benign = {}
for layer in A_benign:
    num = int(len(A_benign[layer]) * top_p)
    sorted_neurons = np.argsort(-A_benign[layer])[:num]
    top_neurons_benign[layer] = [{'neuron': str(neuron), 'weight': str(A_benign[layer][neuron])} for neuron in sorted_neurons]

In [64]:
graph_dir_path = '/Users/haekyu/data/massif/aggregated/panda-armadillo/top-neurons'
file_path = '{}/{}.json'.format(graph_dir_path, 'top-neurons-benign')
with open(file_path, 'w') as f:
    json.dump(top_neurons_benign, f)

## Get the top neurons in **Attacked** graph with different strength

In [65]:
top_p = 0.2
for eps in A_attacked_dict:
    top_neurons_attacked = {}
    A_attacked = A_attacked_dict[eps]
    for layer in A_attacked:
        num = int(len(A_attacked[layer]) * top_p)
        sorted_neurons = np.argsort(-A_attacked[layer])[:num]
        top_neurons_attacked[layer] = [{'neuron': str(neuron), 'weight': str(A_attacked[layer][neuron])} for neuron in sorted_neurons]
        
    graph_dir_path = '/Users/haekyu/data/massif/aggregated/panda-armadillo/top-neurons'
    file_path = '{}/{}-{}.json'.format(graph_dir_path, 'top-neurons-attacked', eps)
    with open(file_path, 'w') as f:
        json.dump(top_neurons_attacked, f)